# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# import numpy and pandas
import numpy as np
import pandas as pd


# Challenge 1 - Independent Sample T-tests

In this challenge, we will be using the Pokemon dataset. Before applying statistical methods to this data, let's first examine the data.

To load the data, run the code below.

In [2]:
# Run this code:

pokemon = pd.read_csv('../pokemon.csv')

Let's start off by looking at the `head` function in the cell below.

In [3]:
# Your code here:

pokemon.columns=[col.replace(' ','_')for col in pokemon]
pokemon.head()

,#,Name,Type_1,Type_2,Total,HP,Attack,Defense,Sp._Atk,Sp._Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


The first thing we would like to do is compare the legendary Pokemon to the regular Pokemon. To do this, we should examine the data further. What is the count of legendary vs. non legendary Pokemons?

In [4]:
# Your code here:
pokemon['Legendary'].value_counts()


False    735
True      65
Name: Legendary, dtype: int64

Compute the mean and standard deviation of the total points for both legendary and non-legendary Pokemon.

In [22]:
# Your code here:
print(pokemon.groupby('Legendary')['Total'].mean())
print(pokemon.groupby('Legendary')['Total'].std())


Legendary
False    417.213605
True     637.384615
Name: Total, dtype: float64
Legendary
False    106.760417
True      60.937389
Name: Total, dtype: float64


The computation of the mean might give us a clue regarding how the statistical test may turn out; However, it certainly does not prove whether there is a significant difference between the two groups.

In the cell below, use the `ttest_ind` function in `scipy.stats` to compare the the total points for legendary and non-legendary Pokemon. Since we do not have any information about the population, assume the variances are not equal.

In [23]:
# Your code here:
from scipy.stats import ttest_ind # independent samples

p_val = ttest_ind(
    pokemon[pokemon['Legendary'] == True]['Total'],
    pokemon[pokemon['Legendary'] == False]['Total'],
    equal_var = False # spread between numbers in data set
)[1]

p_val


9.357954335957446e-47

What do you conclude from this test? Write your conclusions below.

In [7]:
# Your conclusions here:



How about we try to compare the different types of pokemon? In the cell below, list the types of Pokemon from column `Type 1` and the count of each type.

In [8]:
# Your code here:
types = pokemon.pivot_table(
                index=["Type_1"], 
                values=["Total"], 
                aggfunc={"Type_1":len}
            )

types['Type_1'].nlargest(5)
# types.head()

Type_1
Water      112
Normal      98
Grass       70
Bug         69
Psychic     57
Name: Type_1, dtype: int64

Since water is the largest group of Pokemon, compare the mean and standard deviation of water Pokemon to all other Pokemon.

In [25]:
# Your code here:

water = pokemon.query("Type_1 == 'Water'")['Total'].mean() 

non_water = pokemon.query("Type_1 != 'Water'")['Total'].mean()


water_std = pokemon.query("Type_1 == 'Water'")['Total'].std() 

nonwater_std = pokemon.query("Type_1 != 'Water'")['Total'].std()




Perform a hypothesis test comparing the mean of total points for water Pokemon to all non-water Pokemon. Assume the variances are equal. 

In [10]:
# Your code here:
from scipy.stats import ttest_ind

nonwater = pokemon.query("Type_1 != 'Water'")['Total']
water = pokemon.query("Type_1 == 'Water'")['Total']

ttest_ind(
    water,
    nonwater,
    equal_var = True
)




Ttest_indResult(statistic=-0.4418547448849676, pvalue=0.6587140317488793)

Write your conclusion below.

In [11]:
# Your conclusions here:



# Challenge 2 - Matched Pairs Test

In this challenge we will compare dependent samples of data describing our Pokemon. Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. In the cell below, import the `ttest_rel` function from `scipy.stats` and compare the two columns to see if there is a statistically significant difference between them.

In [12]:
# Your code here:
# def interpret(alpha, p_val):
#     print(f"Probability results occurred by chance: {decimal_str(p_val, 30)}.\nIs our p-value less than the alpha? {'Yes. We reject the null hypothesis! Kill it with fire!' if p_val < alpha else 'No. Our null hypothesis is correct.'}")
    
from scipy.stats import ttest_rel

alpha = 0.5

ttest_rel(
    pokemon['Defense'], 
    pokemon['Attack']
) # Compare first die to third die
# p_val

# interpret(alpha, p_val)



Ttest_relResult(statistic=-4.325566393330478, pvalue=1.7140303479358558e-05)

Describe the results of the test in the cell below.

In [13]:
# Your conclusions here:



We are also curious about whether therer is a significant difference between the mean of special defense and the mean of special attack. Perform the hypothesis test in the cell below. 

In [28]:
# Your code here:

sp_att = pokemon['Sp._Atk']
sp_def = pokemon['Sp._Def']

p_val = ttest_rel(sp_att, sp_def)[1]

p_val


0.3933685997548122

Describe the results of the test in the cell below.

In [15]:
# Your conclusions here:



As you may recall, a two sample matched pairs test can also be expressed as a one sample test of the difference between the two dependent columns.

Import the `ttest_1samp` function and perform a one sample t-test of the difference between defense and attack. Test the hypothesis that the difference between the means is zero. Confirm that the results of the test are the same.

In [29]:
# Your code here:

from scipy.stats import ttest_1samp    

expected_mean = 0

p_val_1 = ttest_1samp(
    pokemon['Attack'] - pokemon['Defense'],
    expected_mean
)[1]

p_val
    
    

0.3933685997548122

# Bonus Challenge - The Chi-Square Test

The Chi-Square test is used to determine whether there is a statistically significant difference in frequencies. In other words, we are testing whether there is a relationship between categorical variables or rather when the variables are independent. This test is an alternative to Fisher's exact test and is used in scenarios where sample sizes are larger. However, with a large enough sample size, both tests produce similar results. Read more about the Chi Squared test [here](https://en.wikipedia.org/wiki/Chi-squared_test).

In the cell below, create a contingency table using `pd.crosstab` comparing whether a Pokemon is legenadary or not and whether the Type 1 of a Pokemon is water or not.

In [17]:
# Your code here:



Perform a chi-squared test using the `chi2_contingency` function in `scipy.stats`. You can read the documentation of the function [here](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chi2_contingency.html).

In [18]:
# Your code here:



Based on a 95% confidence, should we reject the null hypothesis?

In [19]:
# Your answer here:

